In [61]:
using DynamicGrids, Crayons, ColorSchemes, Colors, DynamicGridsGtk, Distributions, Random, Statistics, 
    LinearAlgebra

num_species = 10
num_hab = 5
beta_coeff = 0.01*ones(num_species, num_species)
mean_disp_dist = 1
seed_prob = 0.95*ones(num_species)
hab_mat = ones(num_species, num_hab)


death = let betas = beta_coeff, num_species = num_species
    Neighbors(Moore(2)) do data, hood, state, I
        if state == 0
            return(state)
        end
        
        vals = neighbors(hood)       
        abund = [sum(vals .== i) for i in 1:num_species]      
        
        survival_prob = (exp.(-betas[state,:]' * abund))        
        
        if rand() > survival_prob
            0
        else
            state
        end
    end
end
        
birth = Cell() do data, state, I
#     display((data))
#     display(data[:_default_])
# #     display([data[i,j] for i in 1:5, j in 1:5])
# #     println(typeof(values(data)))
#     display(sum(data[:_default_] .== (1)))
# #     display(values(data)[1])

    
    if state == 0
        weights = [sum(data[:_default_] .== [i]) for i in 1:num_species]
        
        display(weights)
        
        wsample(1:10, weights)
#         rand(1:10)
        
        
    else
        state
    end
    
end

birth_step_kernel = let num_species = num_species
    Neighbors(Moore(4)) do data, hood, state, I
        if state == 0
            vals = neighbors(hood)
            weights = [sum(vals .== i) for i in 1:num_species] 
            
            new_seed = wsample(1:10, weights)
            
            if rand() > seed_prob[new_seed]
                0
            else
                new_seed
            end
        else
            state
        end
    end
end

birth_gauss_kernel = let num_species = num_species, mean_disp_dist = mean_disp_dist, seed_prob = seed_prob
    Neighbors(Moore(4)) do data, hood, state, I
        if state == 0
            vals = collect(neighbors(hood))
            dist = collect(distances(hood).^2)
            
            ker = exp.(-dist/2/mean_disp_dist^2)
            weights = [sum(ker[vals .== i]) for i in 1:num_species] 
            
            new_seed = wsample(1:10, weights)
            
            if rand() > seed_prob[new_seed]
                0
            else
                new_seed
            end
        else
            state
        end
    end
end

death_hf = let betas = beta_coeff, num_species = num_species
    Neighbors{:species}(Moore(2)) do data, hood, state, I
        if state == 0
            return(state)
        end
        
        vals = neighbors(hood)       
        abund = [sum(vals .== i) for i in 1:num_species]      
        
        survival_prob = (exp.(-betas[state,:]' * abund))        
        
        if rand() > survival_prob
            0
        else
            state
        end
    end
end
        
birth_g_hf = let num_species = num_species, mean_disp_dist = mean_disp_dist, seed_prob = seed_prob,
    hab_mat = hab_mat
    Neighbors{Tuple{:species, :habitat}, :species}(Moore(4)) do data, hood, (spec, hab), I
        if spec == 0
            vals = collect(neighbors(hood))
            dist = collect(distances(hood).^2)
            
            ker = exp.(-dist/2/mean_disp_dist^2)
            weights = [sum(ker[vals .== i]) for i in 1:num_species] .* hab_mat[:, hab]
            
            new_seed = wsample(1:10, weights)
            
            if rand() > seed_prob[new_seed]
                0
            else
                new_seed
            end
        else
            spec
        end
    end
end
        

Neighbors{Tuple{:species, :habitat},:species}(
    f = var"#129#131"{Int64, Int64, Vector{Float64}, Matrix{Float64}}
    neighborhood = Moore{4, 2, 80, Nothing}
)

In [ ]:
Random.seed!(1)
init = rand(1:10, 5, 5)

output = ArrayOutput(init;
    tspan = 1:20000
)

@time sim!(output, (death, birth_gauss_kernel); boundary = Wrap())

In [ ]:
Random.seed!(1)
init = rand(1:10, 100,100)

output = GifOutput(init; 
    filename="comp_step.gif", 
    tspan=1:200, 
    fps=25, 
    minval = 0, maxval = num_species,
    scheme=ColorSchemes.rainbow,
    zerocolor=RGB24(0.0)
)

@time sim!(output, (death, birth_step_kernel); boundary = Wrap())

In [ ]:
Random.seed!(1)
init = rand(1:10, 500,500)

output = GifOutput(init; 
    filename="comp_gauss.gif", 
    tspan=1:200, 
    fps=25, 
    minval = 0, maxval = num_species,
    scheme=ColorSchemes.rainbow,
    zerocolor=RGB24(0.0)
)

@time sim!(output, (death, birth_gauss_kernel); boundary = Wrap())

In [ ]:
Random.seed!(1)
init = (species = rand(1:10, 100, 100), habitat = rand(1:5, 100, 100))

output = GifOutput(init; 
    filename="comp_gauss.gif", 
    tspan=1:200, 
    fps=25, 
    minval = 0, maxval = num_species,
    scheme=ColorSchemes.rainbow,
    zerocolor=RGB24(0.0)
)

@time sim!(output, (death_hf, birth_g_hf); boundary = Wrap())

In [70]:
Random.seed!(1)
species = rand(1:10, 100, 100)
habitat = rand(1:5, 100, 100)

extent = Extent(; init = (species = species, habitat = habitat), tspan = 1:2)
rules = Ruleset(death_hf, birth_g_hf; boundary = Wrap())

simdata = DynamicGrids.SimData(extent, rules)

t = 0
tmax = 1000
prev_steps = 50 # Previous steps to count in checking distribution
prev_dist = zeros(prev_steps, num_species + 1) # Stores previous 'prev_steps' distributions
threshold = 100*100*0.02 # Threshold for how much the L1 norm should differ before breaking out of the loop

@time while t < tmax
    simdata = step!(simdata)
    
    species_dist = [sum(DynamicGrids.gridview(simdata[:species]) .== i) for i in 0:10]
        
    prev_dist[t % prev_steps + 1, :] = species_dist
    prev_dist_mean = mean(prev_dist; dims = 1)
    
    if t % 10 == 0
        println(norm(mean(prev_dist; dims = 1) - reshape(species_dist, 1, 11), 1))
    end
    
    if norm(mean(prev_dist; dims = 1) - reshape(species_dist, 1, 11), 1) < threshold
        println("Broke out of loop, t = ", t)
        break
    end
    
    t += 1
end

9800.0
7800.000000000002
5800.0
3800.0000000000005
1809.2
578.3999999999999
602.6800000000003
434.7599999999999
498.9600000000001
437.5200000000001
484.03999999999996
671.68
393.31999999999994
443.44000000000005
462.9999999999999
424.8399999999999
476.4000000000001
406.56000000000023
326.2000000000001
449.59999999999997
432.8399999999998
530.04
464.9200000000001
526.0799999999999
435.12
277.12
532.72
515.1599999999999
549.7599999999998
627.04
446.12
524.1599999999999
506.3199999999998
642.2000000000003
687.3200000000002
652.1199999999998
648.9600000000002
489.7199999999999
480.0000000000002
647.4799999999998
580.0400000000001
495.2399999999997
397.6800000000003
533.5600000000002
459.5600000000002
551.4
494.7999999999998
458.27999999999986
615.44
623.1600000000001
447.04
588.9200000000001
598.1199999999999
522.5999999999999
559.56
545.2
415.60000000000014
323.24000000000007
393.96000000000026
479.0800000000001
556.2799999999997
416.3599999999999
445.20000000000005
198.48000000000013
Bro